In [ ]:
import pickle
from transformers import BertTokenizer

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf
#import tensorflow_text

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/he_to_pt',
                               with_info=True,
                               as_supervised=True)

train_examples, test_examples = examples['train'], examples['test']
test_examples

In [ ]:
PAD = 0


class Tokenizer:
    def __init__(self, data_to_fit, isLoad=False):
        """Initilize the tokenizer object

        Args:
            data_to_fit (_type_): the data to fit on the tokenizer / the file path to load if loading is choosen
            isLoad (bool, optional): If the data_to_fit is a file path to load. Defaults to False.
        """
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
            oov_token="[?]",
        )
        if isLoad:  # if load option load the file instead
            self.load(data_to_fit)
            return
        self.tokenizer.fit_on_texts(
            [n.numpy().decode("utf-8") for n in list(data_to_fit.map(lambda x: x))]
        )
        self.tokenizer.index_word[PAD] = "[PAD]"
        self.tokenizer.word_index[self.tokenizer.index_word[PAD]] = PAD
        self.START = self.add_token("[START]")
        self.END = self.add_token("[END]")

    def add_token(self, string):
        index = max(self.tokenizer.index_word.keys()) + 1
        self.tokenizer.index_word[index] = string
        self.tokenizer.word_index[string] = index
        return index

    def tokenize(self, x):
        return tf.ragged.stack(
            list(
                map(
                    lambda x: tf.concat([[self.START], x, [self.END]], 0),
                    self.tokenizer.texts_to_sequences(
                        map(lambda x: x.decode("utf-8"), x.numpy())
                    ),
                )
            )
        )

    def detokenize(self, d):
        return tf.ragged.stack(
            self.tokenizer.sequences_to_texts(map(lambda x: x.numpy(), d))
        )

    def lookup_one(self, x):
        return tf.convert_to_tensor([self.tokenizer.index_word[i] for i in x.numpy()])

    def lookup(self, x):
        return tf.ragged.stack(list(map(lambda indexes: self.lookup_one(indexes), x)))

    def word_count(self):
        return len(self.tokenizer.word_index)

    def save(self, filename: str):
        with open(filename + ".pkl", "wb") as f:  # save the model
            pickle.dump(
                (
                    self.tokenizer.index_word,
                    self.tokenizer.word_index,
                    self.START,
                    self.END,
                ),
                f,
            )

    def load(self, file: str):
        with open(file, "rb") as f:  # load the model
            (
                self.tokenizer.index_word,
                self.tokenizer.word_index,
                self.START,
                self.END,
            ) = pickle.load(f)

In [ ]:
tokenizer_he = Tokenizer(train_examples.map(lambda he, pt: he))
tokenizer_pt = Tokenizer(train_examples.map(lambda he, pt: pt))


In [ ]:
for he_examples, pt_examples in train_examples.batch(3).take(1):
  print('> Examples in Hebrew:')
  for he in he_examples.numpy():
    print(he.decode('utf-8'))
  print()

  print('> Examples in Portuguese:')
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))

In [ ]:
print('> This is a batch of strings:')
for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))


In [ ]:
encoded = tokenizer_pt.tokenize(pt_examples)

print('> This is a padded-batch of token IDs:')
for row in encoded:
  print(row)

In [ ]:
round_trip = tokenizer_pt.detokenize(encoded)
print('> This is human-readable text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))


In [ ]:
print('> This is the text split into tokens:')
tokens = tokenizer_pt.lookup(encoded)
[[token.decode('utf-8') for token in seq] for seq in tokens.numpy()]

In [ ]:
lengths = []

for he_examples, pt_examples in train_examples.batch(1024):
  he_tokens = tokenizer_he.tokenize(he_examples)
  lengths.append(he_tokens.row_lengths())

  pt_tokens = tokenizer_pt.tokenize(pt_examples)
  lengths.append(pt_tokens.row_lengths())
  print('.', end='', flush=True)

In [ ]:
all_lengths = np.concatenate(lengths)

plt.hist(all_lengths, np.linspace(0, 200, 101))
plt.ylim(plt.ylim())
max_length = max(all_lengths)
plt.plot([max_length, max_length], plt.ylim())
plt.title(f'Maximum tokens per example: {max_length}');
plt.xlabel('Tokens Per Example')
plt.ylabel('Exampels')

In [ ]:
MAX_TOKENS = 128
def prepare_batch(he, pt):

    he = tokenizer_he.tokenize(he)     # Output is ragged.
    he = he[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    he = he.to_tensor()

    pt = tokenizer_pt.tokenize(pt)
    pt = pt[:, :(MAX_TOKENS+1)]
    pt_inputs = pt[:, :-1].to_tensor()  # Drop the [END] tokens
    pt_labels = pt[:, 1:].to_tensor()  # Drop the [CLS] tokens

    return (he,pt_inputs), pt_labels

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
def make_batches(ds):
  return (
    tf.data.experimental.from_list(
      list(
        map(lambda x: prepare_batch(*x),ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)))
        )
        #.map(lambda he,pt_inputs, pt_labels: ((he[0],pt_inputs[0]), pt_labels[0]), tf.data.AUTOTUNE)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

In [ ]:
# Create training and validation set batches.
test_batches = make_batches(test_examples)
train_batches = make_batches(train_examples)


In [ ]:
for (he, pt), pt_labels in train_batches.take(1):
  continue

print(he.shape)
print(pt.shape)
print(pt_labels.shape)

In [ ]:
print(pt[0][:10])
print(pt_labels[0][:10])

In [ ]:
print(tokenizer_he.detokenize(he[:1]))

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
pos_encoding = positional_encoding(length=2048, depth=512)

# Check the shape.
print(pos_encoding.shape)

# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [ ]:
pos_encoding/=tf.norm(pos_encoding, axis=1, keepdims=True)
p = pos_encoding[1000]
dots = tf.einsum('pd,d -> p', pos_encoding, p)
plt.subplot(2,1,1)
plt.plot(dots)
plt.ylim([0,1])
plt.plot([950, 950, float('nan'), 1050, 1050],
         [0,1,float('nan'),0,1], color='k', label='Zoom')
plt.legend()
plt.subplot(2,1,2)
plt.plot(dots)
plt.xlim([950, 1050])
plt.ylim([0,1])

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
embed_he = PositionalEmbedding(vocab_size=tokenizer_he.word_count() , d_model=512)
embed_pt = PositionalEmbedding(vocab_size=tokenizer_pt.word_count() , d_model=512)

he_emb = embed_he(he)
pt_emb = embed_pt(pt)

In [ ]:
print(embed_he.embedding.output_dim)

In [ ]:
pt_emb._keras_mask
pt_emb.shape

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
sample_ca = CrossAttention(num_heads=2, key_dim=512)

print(he_emb.shape)
print(pt_emb.shape)
print(sample_ca(pt_emb, he_emb).shape)

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_gsa = GlobalSelfAttention(num_heads=2, key_dim=512)

print(he_emb.shape)
print(sample_gsa(he_emb).shape)

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_csa = CausalSelfAttention(num_heads=2, key_dim=512)

print(pt_emb.shape)
print(sample_csa(pt_emb).shape)

In [ ]:
out1 = sample_csa(embed_pt(pt[:, :3])) 
out2 = sample_csa(embed_pt(pt))[:, :3]

tf.reduce_max(abs(out1 - out2)).numpy()

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

In [ ]:
sample_ffn = FeedForward(512, 2048)

print(pt_emb.shape)
print(sample_ffn(pt_emb).shape)

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
sample_encoder_layer = EncoderLayer(d_model=512, num_heads=8, dff=2048)

print(he_emb.shape)
print(sample_encoder_layer(he_emb).shape)

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
# Instantiate the encoder.
sample_encoder = Encoder(num_layers=4,
                         d_model=512,
                         num_heads=8,
                         dff=2048,
                         vocab_size=tokenizer_he.word_count())

sample_encoder_output = sample_encoder(he, training=False)

# Print the shape.
print(he.shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
sample_decoder_layer = DecoderLayer(d_model=512, num_heads=8, dff=2048)

sample_decoder_layer_output = sample_decoder_layer(
    x=pt_emb, context=he_emb)

print(pt_emb.shape)
print(he_emb.shape)
print(sample_decoder_layer_output.shape)  # `(batch_size, seq_len, d_model)`

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
# Instantiate the decoder.
sample_decoder = Decoder(num_layers=4,
                         d_model=512,
                         num_heads=8,
                         dff=2048,
                         vocab_size=tokenizer_pt.word_count())
output = sample_decoder(
    x=pt,
    context=he_emb)

# Print the shapes.
print(pt.shape)
print(he_emb.shape)
print(output.shape)

In [ ]:
sample_decoder.last_attn_scores.shape  # (batch, heads, target_seq, input_seq)

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:
num_layers = 1
d_model = 64
dff = 512
num_heads = 4
dropout_rate = 0.1

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer_he.word_count(),
    target_vocab_size=tokenizer_pt.word_count(),
    dropout_rate=dropout_rate)

In [ ]:
output = transformer((he, pt))

print(pt.shape)
print(he.shape)
print(output.shape)

In [ ]:
attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

In [ ]:
transformer.summary()

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
plt.plot(learning_rate(tf.range(40000, dtype=tf.float32)))
plt.ylabel('Learning Rate')
plt.xlabel('Train Step')

In [ ]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)  # compute loss

  mask = tf.cast(mask, dtype=loss.dtype) 
  loss *= mask  # aplly mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss

def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)  # get the prediction
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask  # apply mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)  
# in the mask the relevant values are 1 and the rest 0.
# So the sum of the mask is the number of values

In [ ]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [ ]:
transformer.fit(train_batches,
                epochs=1,
                validation_data=test_batches)

In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizer_he,tokenizer_pt, transformer):
    self.tokenizer_he = tokenizer_he
    self.tokenizer_pt = tokenizer_pt
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):
    # The input sentence is Hebrew, hence adding the `[START]` and `[END]` tokens.
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizer_he.tokenize(sentence).to_tensor()

    encoder_input = sentence

    # As the output language is Portuguese, initialize the output with the
    # Portuguese `[START]` token.
    start_end = self.tokenizer_pt.tokenize(tf.convert_to_tensor(['']))[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions= self.transformer([encoder_input, output], training=False)
      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1,output_type=tf.int32)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    text = tokenizer_pt.detokenize(output)[0]  # Shape: `()`.

    tokens = tokenizer_pt.lookup(output)[0]

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    return text, tokens, attention_weights

In [ ]:
translator = Translator(tokenizer_he,tokenizer_pt, transformer)

In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
sentence = 'ילדים אוהבים כיסאות.'
ground_truth = 'eu gosto de comer sorvete.'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

In [ ]:
def plot_attention_head(in_tokens, translated_tokens, attention):
  # The model didn't generate `<START>` in the output. Skip it.
  translated_tokens = translated_tokens[1:]

  ax = plt.gca()
  ax.matshow(attention)
  ax.set_xticks(range(len(in_tokens)))
  ax.set_yticks(range(len(translated_tokens)))

  labels = [label.decode('utf-8') for label in in_tokens.numpy()]
  ax.set_xticklabels(
      labels, rotation=90)

  labels = [label.decode('utf-8') for label in translated_tokens.numpy()]
  ax.set_yticklabels(labels)

In [ ]:
head = 0
# Shape: `(batch=1, num_heads, seq_len_q, seq_len_k)`.
attention_heads = tf.squeeze(attention_weights, 0)
attention = attention_heads[head]
attention.shape

In [ ]:
in_tokens = tf.convert_to_tensor([sentence])
in_tokens = tokenizer_he.tokenize(in_tokens).to_tensor()
in_tokens = tokenizer_he.lookup(in_tokens)[0]
in_tokens

In [ ]:
translated_tokens

In [ ]:
plot_attention_head(in_tokens, translated_tokens, attention)

In [ ]:
def plot_attention_weights(sentence, translated_tokens, attention_heads):
  in_tokens = tf.convert_to_tensor([sentence])
  in_tokens = tokenizer_he.tokenize(in_tokens).to_tensor()
  in_tokens = tokenizer_he.lookup(in_tokens)[0]

  fig = plt.figure(figsize=(16, 8))

  for h, head in enumerate(attention_heads):
    ax = fig.add_subplot(2, 4, h+1)

    plot_attention_head(in_tokens, translated_tokens, head)

    ax.set_xlabel(f'Head {h+1}')

  plt.tight_layout()
  plt.show()

In [ ]:
plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0])